In [2]:
import os

from folioclient import FolioClient

okapi_url = os.getenv("OKAPI_URL")
user = os.getenv("OKAPI_USER")
password = os.getenv("OKAPI_PASSWORD")
tenant = os.getenv("TENANT")

folio_client = FolioClient(
    okapi_url,
    tenant,
    user,
    password)

In [ ]:
query_params = {'fiscalYearId': '200bfabe-07c7-4deb-b54e-99d64a3435cb'}

In [7]:
response = folio_client.folio_get("/finance/transactions", query="?query=((fiscalYearId=200bfabe-07c7-4deb-b54e-99d64a3435cb and (fromFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411 or toFundId=3eb86c5f-c77b-4cc9-8f29-7de7ce313411)) and source==\"Invoice\") sortby transactionDate/sort.descending")

In [8]:
print(response)

{'transactions': [{'id': '3162c4f8-4cfa-4717-869b-10a05fd4b10f', '_version': 1, 'amount': 27.28, 'currency': 'USD', 'fiscalYearId': '200bfabe-07c7-4deb-b54e-99d64a3435cb', 'fromFundId': '3eb86c5f-c77b-4cc9-8f29-7de7ce313411', 'paymentEncumbranceId': '71d22ab7-0ad8-4be9-84c3-b53002fdc1c0', 'source': 'Invoice', 'sourceInvoiceId': '0db6930a-0654-45dd-ae65-86fa915b18a3', 'sourceInvoiceLineId': '0b9ae0fa-792f-4db2-9254-648dfb185bb8', 'transactionType': 'Payment', 'metadata': {'createdDate': '2024-07-24T01:03:08.381+00:00', 'createdByUserId': '07e8be0e-1cf5-5061-b5b3-4517d9a74d77', 'updatedDate': '2024-07-24T01:03:08.381+00:00', 'updatedByUserId': '07e8be0e-1cf5-5061-b5b3-4517d9a74d77'}}, {'id': 'f053724f-5c4d-49c4-ab07-9dc300f9d1a6', '_version': 1, 'amount': 91.45, 'currency': 'USD', 'fiscalYearId': '200bfabe-07c7-4deb-b54e-99d64a3435cb', 'fromFundId': '3eb86c5f-c77b-4cc9-8f29-7de7ce313411', 'paymentEncumbranceId': '369fba7d-3f82-4aff-b00b-f1a7086df7a6', 'source': 'Invoice', 'sourceInvoiceI